<a href="https://colab.research.google.com/github/ostarkweather/ds-content-interactive-jupyterlab-tutorial/blob/master/XSLT_Result_Exploration_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up Google Colab Workspace

In [1]:
# Install packages
!pip install Dash
!pip install dash-bootstrap-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 4.1 MB/s eta 0:00:00


In [2]:
# import libraries
import gspread #gspread is used to login to Google
import pandas as pd
import numpy as np
import plotly.express as px
from dash import html, dcc, Dash, callback
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from google.auth import default
from google.colab import auth

In [3]:
# Authenticate Google Access
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


# Import CSV files into Dataframes

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Folder path used to import all files
folder_path = '/content/drive/MyDrive/_Transgender Media Portal Shared Folder/TOOL_Data Analysis/Python Files/MediaWorks/Source Data - XSLT Generated CSVs/'

## Import Media Works Dataframe

In [6]:
df_media_works = pd.read_csv(folder_path + 'Media Works.csv')
df_media_works = df_media_works.drop(df_media_works.index[[0,1,2,3]]) #drop the first 3 columns (we're already made row 0 the headers, then we need to get rid of the examples)
df_media_works.head()

,ID,Title_of_Work,Alternate_Titles,Country,Release_Year,Series_End_Year,Is_an_Episode_of,Season.Episode,Runtime_Minutes,Primary_Description_Text,...,References,Alternate_Description_1_Text,Alternate_Description_1_Source,Alternate_Description_2_Text,Alternate_Description_2_Source,Alternate_Description_3_Text,Alternate_Description_3_Source,Alternate_Description_4_Text,Alternate_Description_4_Source,Primary_Image
4,WORK_000001,Scent uVa Butch,NaN,United States of America,1998,NaN,NaN,NaN,35,Scent uVa Butch is a documentary exploring the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,WORK_000002,Strait,NaN,United States of America,1996,NaN,NaN,NaN,12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,WORK_000003,we got moves you ain't even heard of (part one),NaN,United States of America,1999,NaN,NaN,NaN,11,We Got Moves You Ain’t Even Heard Of (Part One...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMAG_000422
7,WORK_000004,The Changes Within,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,WORK_000005,Butch Body Blues,NaN,United States of America,2001,NaN,NaN,NaN,12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create Additional Tables

## Release Year

In [7]:
# prompt: create a list starting at the smallest value in Release_Year from df and ending at the maximum Release_Year

# Assuming 'Release_Year' column exists and contains numeric data.
# Convert 'Release_Year' to numeric if it's not already
df_media_works['Release_Year'] = pd.to_numeric(df_media_works['Release_Year'], errors='coerce')

min_year = df_media_works['Release_Year'].min()
max_year = df_media_works['Release_Year'].max()

release_years_list = list(range(int(min_year), int(max_year) + 1))

df_release_year = pd.DataFrame({'Release_Year': release_years_list})
df_release_year.head()

,Release_Year
0,1901
1,1902
2,1903
3,1904
4,1905


In [8]:
'''
df_genre = pd.read_csv(folder_path + 'join-Media_Works-Genre.csv')
df_genre.head()
'''

"\ndf_genre = pd.read_csv(folder_path + 'join-Media_Works-Genre.csv')\ndf_genre.head()\n"

# Join Dataframes Together

## Drop Columns from the df_media_works dataframe

Dropping any columns not used in reporting should help with performance

In [9]:
df_media_works_transformed = df_media_works[['ID', 'Title_of_Work','Release_Year', 'Runtime_Minutes', 'BW_and_or_Colour', 'Media_Type']].copy()
df_media_works_transformed.head()

,ID,Title_of_Work,Release_Year,Runtime_Minutes,BW_and_or_Colour,Media_Type
4,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video
5,WORK_000002,Strait,1996.0,12,Colour,film/video
6,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video
7,WORK_000004,The Changes Within,2001.0,NaN,NaN,NaN
8,WORK_000005,Butch Body Blues,2001.0,12,NaN,film/video


## Join Creator Identities

### Import and Join Creator Gender Closed Vocab

In [10]:
df_join_Media_Works_Creator_Gender_Closed_Vocab = pd.read_csv(folder_path + 'join-Media_Works-Creator_Gender_Closed_Vocab.csv')
# df_join_Media_Works_Creator_Gender_Closed_Vocab.head()

In [11]:
# Merge the dataframes
df_media_works_transformed = pd.merge(df_media_works_transformed, df_join_Media_Works_Creator_Gender_Closed_Vocab, left_on='ID', right_on='Media_Works', how='left')

# Discard the columns that were added through the merge, but we don't need
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Media_Works')]]
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Vocab_ID')]]

# See Results
df_media_works_transformed.head()

,ID,Title_of_Work,Release_Year,Runtime_Minutes,BW_and_or_Colour,Media_Type,Creator_Gender_Closed_Vocab
0,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN
1,WORK_000002,Strait,1996.0,12,Colour,film/video,NaN
2,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer
3,WORK_000004,The Changes Within,2001.0,NaN,NaN,NaN,NaN
4,WORK_000005,Butch Body Blues,2001.0,12,NaN,film/video,NaN


### Join Creator Race & Ethnicity

In [12]:
df_join_creator_Race_Ethnicity_Closed_Vocab = pd.read_csv(folder_path + 'join-Media_Works-Creator_Race_Ethnicity_Closed_Vocab.csv')
df_join_creator_Race_Ethnicity_Closed_Vocab.head()

# Merge the dataframes
df_media_works_transformed = pd.merge(df_media_works_transformed, df_join_creator_Race_Ethnicity_Closed_Vocab, left_on='ID', right_on='Media_Works', how='left')

# Discard the columns that were added through the merge, but we don't need
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Media_Works')]]
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Vocab_ID')]]

df_media_works_transformed.head()

,ID,Title_of_Work,Release_Year,Runtime_Minutes,BW_and_or_Colour,Media_Type,Creator_Gender_Closed_Vocab,Creator_Race_Ethnicity_Closed_Vocab
0,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN,South Asian
1,WORK_000002,Strait,1996.0,12,Colour,film/video,NaN,NaN
2,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer,East Asian
3,WORK_000004,The Changes Within,2001.0,NaN,NaN,NaN,NaN,NaN
4,WORK_000005,Butch Body Blues,2001.0,12,NaN,film/video,NaN,NaN


## Join Multi-select Vocabs

### Join Genre

In [13]:
df_Genre = pd.read_csv(folder_path + 'join-Media_Works-Genre.csv')
df_Genre.head()

# Merge dataframe
df_media_works_transformed = pd.merge(df_media_works_transformed, df_Genre, left_on='ID', right_on='Media_Works', how='left')


# Discard the columns that were added through the merge, but we don't need
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Media_Works')]]
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Vocab_ID')]]

#view results
df_media_works_transformed.head()

,ID,Title_of_Work,Release_Year,Runtime_Minutes,BW_and_or_Colour,Media_Type,Creator_Gender_Closed_Vocab,Creator_Race_Ethnicity_Closed_Vocab,Genre
0,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN,South Asian,documentary
1,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN,South Asian,short (less than 45 min)
2,WORK_000002,Strait,1996.0,12,Colour,film/video,NaN,NaN,short (less than 45 min)
3,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer,East Asian,comedy
4,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer,East Asian,fantasy


### Join Country

In [14]:
df_Country = pd.read_csv(folder_path + 'join-Media_Works-Country.csv')
df_Country.head()

# Merge dataframe
df_media_works_transformed = pd.merge(df_media_works_transformed, df_Country, left_on='ID', right_on='Media_Works', how='left')


# Discard the columns that were added through the merge, but we don't need
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Media_Works')]]
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Vocab_ID')]]

df_media_works_transformed.head()

,ID,Title_of_Work,Release_Year,Runtime_Minutes,BW_and_or_Colour,Media_Type,Creator_Gender_Closed_Vocab,Creator_Race_Ethnicity_Closed_Vocab,Genre,Country
0,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN,South Asian,documentary,United States of America
1,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN,South Asian,short (less than 45 min),United States of America
2,WORK_000002,Strait,1996.0,12,Colour,film/video,NaN,NaN,short (less than 45 min),United States of America
3,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer,East Asian,comedy,United States of America
4,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer,East Asian,fantasy,United States of America


### Join Filming Format

In [15]:
df_Filming_Format = pd.read_csv(folder_path + 'join-Media_Works-Filming_Format.csv')
df_Filming_Format.head()

# Merge dataframe
df_media_works_transformed = pd.merge(df_media_works_transformed, df_Filming_Format, left_on='ID', right_on='Media_Works', how='left')

# Discard the columns that were added through the merge, but we don't need
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Media_Works')]]
df_media_works_transformed = df_media_works_transformed[[c for c in df_media_works_transformed.columns if not c.startswith('Vocab_ID')]]

df_media_works_transformed.head()

,ID,Title_of_Work,Release_Year,Runtime_Minutes,BW_and_or_Colour,Media_Type,Creator_Gender_Closed_Vocab,Creator_Race_Ethnicity_Closed_Vocab,Genre,Country,Filming_Format
0,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN,South Asian,documentary,United States of America,Video
1,WORK_000001,Scent uVa Butch,1998.0,35,Colour,film/video,NaN,South Asian,short (less than 45 min),United States of America,Video
2,WORK_000002,Strait,1996.0,12,Colour,film/video,NaN,NaN,short (less than 45 min),United States of America,16mm
3,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer,East Asian,comedy,United States of America,other celluloid film format
4,WORK_000003,we got moves you ain't even heard of (part one),1999.0,11,B/W,film/video,genderqueer,East Asian,fantasy,United States of America,other celluloid film format


# Dash App

The thing to keep in mind, is that this graph is focused on films. This means if a film has multiple filmmakers one of who is non-binary and one of whom is cis. The film will be counted under both cis and trans.

In [19]:
external_stylesheets = [dbc.themes.FLATLY]

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    dbc.Row(html.H1("Media Works Categorical Data")),
    dbc.Row(
      dcc.RadioItems(
          options=[
            {'label':'Media Type', 'value':'Media_Type'},
            {'label':'Creator Gender - Closed Vocab', 'value':'Creator_Gender_Closed_Vocab'},
            {'label':'Creator Race & Ethnicity - Closed Vocab', 'value':'Creator_Race_Ethnicity_Closed_Vocab'},
            {'label':'Country', 'value':'Country'},
            {'label':'Genre', 'value':'Genre'},
            {'label':'Colour or Black & White', 'value':'BW_and_or_Colour'},
            {'label':'Filming Format', 'value':'Filming_Format'}
          ],
          id='count-dropdown',
          value='Media_Type',
          inline=True, inputStyle={"margin-left": "20px"}
      )
    ),
    dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Genre'].dropna().unique(),
                id='genre-dropdown',
                placeholder='Select a Genre'
            ), width=2
        ),
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Media_Type'].dropna().unique(),
                id='media-dropdown',
                placeholder='Select a Media Type'
              ), width=2
        ),
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Country'].dropna().unique(),
                id='country-dropdown',
                placeholder='Select a Country'
              ), width=2
        ),
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Creator_Gender_Closed_Vocab'].dropna().unique(),
                id='gender-dropdown',
                placeholder='Select a Creator Gender'
              ), width=2
        )
    ]),
    dbc.Row(
        dcc.Graph(id='media_bar_graph')
    )
])

@callback(
    Output('media_bar_graph', 'figure'),
    [Input('count-dropdown', 'value'),
    Input('genre-dropdown', 'value'),
    Input('media-dropdown', 'value'),
    Input('country-dropdown', 'value'),
    Input('gender-dropdown', 'value')]
)

def update_graph(selected_count, selected_genre, selected_media_type, selected_country, selected_gender):

    group_by_field = selected_count

    filtered_df = df_media_works_transformed
    if selected_genre:
        filtered_df = filtered_df[filtered_df['Genre'] == selected_genre]
    if selected_media_type:
        filtered_df = filtered_df[filtered_df['Media_Type'] == selected_media_type]
    if selected_country:
        filtered_df = filtered_df[filtered_df['Country'] == selected_country]
    if selected_gender:
        filtered_df = filtered_df[filtered_df['Creator_Gender_Closed_Vocab'] == selected_gender]

    distinct_id_counts = filtered_df.groupby(group_by_field)['ID'].nunique()
    fig = px.bar(distinct_id_counts, x=distinct_id_counts.index, y=distinct_id_counts.values,
                 labels={'x': 'Creator Gender', 'y': 'Number of Unique IDs'})
    fig.update_layout(
        title_text=f'Number of Unique IDs per Creator Gender (Genre: {selected_genre}, Media Type: {selected_media_type})',
        xaxis={'categoryorder':'total descending'})
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

## In Progress Dash App

In [20]:
external_stylesheets = [dbc.themes.FLATLY]

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    dbc.Row(html.H1("Media Works Categorical Data")),
    dbc.Row(
      dcc.RadioItems(
          options=[
            {'label':'Media Type', 'value':'Media_Type'},
            {'label':'Creator Gender - Closed Vocab', 'value':'Creator_Gender_Closed_Vocab'},
            {'label':'Creator Race & Ethnicity - Closed Vocab', 'value':'Creator_Race_Ethnicity_Closed_Vocab'},
            {'label':'Country', 'value':'Country'},
            {'label':'Genre', 'value':'Genre'},
            {'label':'Colour or Black & White', 'value':'BW_and_or_Colour'},
            {'label':'Filming Format', 'value':'Filming_Format'}
          ],
          id='count-dropdown',
          value='Media_Type',
          inline=True, inputStyle={"margin-left": "20px"}
      )
    ),
    dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Genre'].dropna().unique(),
                id='genre-dropdown',
                placeholder='Select a Genre'
            ), width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Media_Type'].dropna().unique(),
                id='media-dropdown',
                placeholder='Select a Media Type'
              ), width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Country'].dropna().unique(),
                id='country-dropdown',
                placeholder='Select a Country'
              ), width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Creator_Gender_Closed_Vocab'].dropna().unique(),
                id='gender-dropdown',
                placeholder='Select a Creator Gender'
              ), width=3
        )
    ]),
    dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                df_media_works_transformed['Creator_Race_Ethnicity_Closed_Vocab'].dropna().unique(),
                id='race-ethnicity-dropdown',
                placeholder='Select a Creator Gender'
              ), width=3
        )
    ]),
    dbc.Row(
        dcc.Graph(id='media_bar_graph')
    )
])

@callback(
    Output('media_bar_graph', 'figure'),
    [Input('count-dropdown', 'value'),
    Input('genre-dropdown', 'value'),
    Input('media-dropdown', 'value'),
    Input('country-dropdown', 'value'),
    Input('gender-dropdown', 'value'),
    Input('race-ethnicity-dropdown', 'value')]
)

def update_graph(selected_count, selected_genre, selected_media_type, selected_country, selected_gender, selected_race_ethnicity):

    group_by_field = selected_count

    filtered_df = df_media_works_transformed
    if selected_genre:
        filtered_df = filtered_df[filtered_df['Genre'] == selected_genre]
    if selected_media_type:
        filtered_df = filtered_df[filtered_df['Media_Type'] == selected_media_type]
    if selected_country:
        filtered_df = filtered_df[filtered_df['Country'] == selected_country]
    if selected_gender:
        filtered_df = filtered_df[filtered_df['Creator_Gender_Closed_Vocab'] == selected_gender]
    if selected_gender:
        filtered_df = filtered_df[filtered_df['Creator_Race_Ethnicity_Closed_Vocab'] == selected_race_ethnicity]

    distinct_id_counts = filtered_df.groupby(group_by_field)['ID'].nunique()
    fig = px.bar(distinct_id_counts, x=distinct_id_counts.index, y=distinct_id_counts.values,
                 labels={'x': 'Creator Gender', 'y': 'Number of Unique IDs'})
    fig.update_layout(
        title_text=f'Number of Unique IDs per Creator Gender (Genre: {selected_genre}, Media Type: {selected_media_type})',
        xaxis={'categoryorder':'total descending'})
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

# Alternate Versions

In [18]:
# this version of the app gives you a "Select All"

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Media Works Bar Graph"),
    dcc.RadioItems(options=[
        {'label':'Media Type', 'value':'Media_Type'},
        {'label':'Creator Gender - Closed Vocab', 'value':'Creator_Gender_Closed_Vocab'},
        {'label':'Creator Race & Ethnicity - Closed Vocab', 'value':'Creator_Race_Ethnicity_Closed_Vocab'},
        {'label':'Country', 'value':'Country'},
        {'label':'Genre', 'value':'Genre'},
        {'label':'Colour or Black & White', 'value':'BW_and_or_Colour'},
        {'label':'Filming Format', 'value':'Filming_Format'}
      ],
      id='count-dropdown',
      value='Media_Type',
      inline=True
    ),
    dcc.Dropdown(
        id='genre-dropdown',
        options=[{'label': 'All Genres', 'value': 'All'}] + [{'label': genre, 'value': genre} for genre in df_media_works_transformed['Genre'].dropna().unique()], # Get a unique list of all values. You have to null values cause it to fail, so you have to get rid of them. Then add an "all" option
        placeholder='Select a Genre',
        value = 'All'),
    dcc.Dropdown(
        id='media-dropdown',
        options=[{'label': 'All Media Types', 'value': 'All'}] + [{'label': media_type, 'value': media_type} for media_type in df_media_works_transformed['Media_Type'].dropna().unique()],
        placeholder='Select a Media Type',
        value = 'All'),
    dcc.Graph(id='media_bar_graph')
])

@callback(
    Output('media_bar_graph', 'figure'),
    [Input('count-dropdown', 'value'),
    Input('genre-dropdown', 'value'),
    Input('media-dropdown', 'value')]
)
def update_graph(selected_count, selected_genre, selected_media_type):

    group_by_field = selected_count

    filtered_df = df_media_works_transformed
    if selected_genre != 'All':
        filtered_df = filtered_df[filtered_df['Genre'] == selected_genre]
    if selected_media_type != 'All':
        filtered_df = filtered_df[filtered_df['Media_Type'] == selected_media_type]

    distinct_id_counts = filtered_df.groupby(group_by_field)['ID'].nunique()
    fig = px.bar(distinct_id_counts, x=distinct_id_counts.index, y=distinct_id_counts.values,
                 labels={'x': 'Creator Gender', 'y': 'Number of Unique IDs'})
    fig.update_layout(title_text=f'Number of Unique IDs per Creator Gender (Genre: {selected_genre}, Media Type: {selected_media_type})')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>